In [130]:
import torch
import torch.nn.functional as F
words = open('../names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {ch: i+1  for i, ch in enumerate(chars)}
stoi['.'] = 0
itos = {i: ch for ch, i, in stoi.items()}

In [131]:
# create the training set of bigrams and targets (x, y)

xs, ys = [], []

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    xs.append(stoi[ch1])
    ys.append(stoi[ch2])
    #print(f'{ch1}, {ch2}')
    
xsT = torch.tensor(xs)
ysT = torch.tensor(ys)

xsT, ysT


(tensor([ 0,  5, 13, 13,  1]), tensor([ 5, 13, 13,  1,  0]))

In [132]:
# forward pass
g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(xsT, num_classes=27).float()

# Initialize weights
W = torch.randn(27, 27, requires_grad=True)
logits = xenc @ W # log-counts
counts = logits.exp()  # Equivalent to N frequency matrix
probs = counts / counts.sum(dim=1, keepdim=True)
# plt.imshow(xenc, cmap='Blues')

In [133]:
# compute the loss
loss = -probs[torch.arange(len(ysT)), ysT].log().sum()

In [134]:
print(f'{loss.item()=}')

loss.item()=20.107805252075195


In [135]:
# Backward pass
W.grad = None # zero the gradients
loss.backward()

In [136]:
# update the weights
learning_rate = 8
with torch.no_grad():
   W -= learning_rate * W.grad

In [137]:
print(f'{loss=}')

loss=tensor(20.1078, grad_fn=<NegBackward0>)


# All together 

In [138]:
import torch.nn.functional as F


# Example word list for testing
words = open('../names.txt', 'r').read().splitlines()
# Initialize the stoi dictionary (character to index mapping)
chars = sorted(list(set(''.join(words))))
stoi = {ch: i+1 for i, ch in enumerate(chars)}
stoi['.'] = 0
itos = {i: ch for ch, i, in stoi.items()}

# Create the training set of bigrams and targets (x, y)
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])
        #print(f'{ch1}, {ch2}')

xsT = torch.tensor(xs)
ysT = torch.tensor(ys)

# Forward pass
g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(xsT, num_classes=27).float()

# Initialize weights
W = torch.randn(27, 27, requires_grad=True)

# Define a learning rate
learning_rate = 8

In [139]:


# Training loop (for illustration, using only 2 iterations)
for i in range(400):
    # Compute logits
    logits = xenc @ W  # it will output a row within W whichis the same a s a ROW in makemore counting bigrams version
    
    # Compute probabilities
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    
    # Compute the loss
    # 0.01 * (W**2).sum() is the regularization term that pushes towards a uniform distribution it is needed to prevent overfittting and smooting the moel
    loss = -probs[torch.arange(len(ysT)), ysT].log().mean() + 0.01 * (W**2).mean() 
    # print(f'Iteration {i+1}, {loss.item()=}')
    
    # Backward pass
    W.grad = None  # zero the gradients
    loss.backward()
    
    # Update the weights
    with torch.no_grad():
        W -= learning_rate * W.grad


In [140]:
logits = xenc @ W  # it will output a row within W whichis the same a s a ROW in makemore counting bigrams version
    
# Compute probabilities
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)
probs

tensor([[0.0020, 0.1372, 0.0405,  ..., 0.0049, 0.0166, 0.0288],
        [0.1940, 0.0327, 0.0066,  ..., 0.0068, 0.0517, 0.0080],
        [0.0721, 0.3837, 0.0112,  ..., 0.0049, 0.0360, 0.0039],
        ...,
        [0.2030, 0.2169, 0.0033,  ..., 0.0017, 0.0059, 0.0050],
        [0.0534, 0.3424, 0.0085,  ..., 0.0054, 0.0207, 0.0177],
        [0.0848, 0.1330, 0.0243,  ..., 0.0205, 0.0050, 0.0047]],
       grad_fn=<DivBackward0>)

In [141]:
# sample from the model

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ---------------
    # BEFORE:
    # p = P[ix]
    # ---------------
    # NOW: 
    with torch.no_grad():
      xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
      logits = xenc @ W # pre3dict the logs-count
      counts = logits.exp()
      probs = counts / counts.sum(dim=1, keepdim=True)
      
      ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
        break
  print(''.join(out))
      
    

cexbr.
momasurailezityha.
konimittain.
llayn.
ka.
